In [1]:
#!/usr/bin/env python
# generic scientific/ipython header
from __future__ import print_function
from __future__ import division
import os, sys
import copy
import numpy as np

In [2]:
import openeye.oechem as oechem

### Load lib for chemical environments

Note full path to the library over in the smarty git tree

In [3]:
import imp
smirky = imp.load_source('environment','/Users/bayly/BaylyData/collaborations/sabbatical/smarty/smarty/environment.py')

### Chemical groups we might use

In [4]:
chemGroups = [ ('ewg1', '[#7,#8!-1,#16!-1,F,Cl,Br,I]'),
               ('ewg1di', '[#7!X1,#8!X1,#16!X1]') ]

### torsions from atom and bond sublists

In [5]:
# Sublists
A_any = [None,None]
A_C = [ ['#6'], None]
A_Ctet = [ ['#6X4'], None]
A_O = [ ['#8'], None]
A_OEth = [ ['#8X2'], ['H0','+0']]
A_OH = [ ['#8X2'], ['H1','+0'] ]
A_H = [ ['H'], None]
Bsngl = [ ['-'], None]
BnoRing = [ ['!@'], None]
B_any = [ None, None]

In [6]:
# specific torsions from sublists
torsionEnv2 = smirky.TorsionChemicalEnvironment( A_Ctet,B_any,A_Ctet,Bsngl,A_OH,B_any,A_any )
torsionEnv3 = smirky.TorsionChemicalEnvironment( A_OEth,B_any,A_Ctet,Bsngl,A_OH,B_any,A_any )
torsionEnv4 = smirky.TorsionChemicalEnvironment( A_OEth,B_any,A_Ctet,Bsngl,A_O,B_any,A_any )
print( torsionEnv4.asSMIRKS() )

[#8X2;H0;+0:1]~[#6X4:2]-[#8:3]~[*:4]


In [7]:
# specific angles from sublists 
angEnv1 = smirky.AngleChemicalEnvironment( A_OEth,B_any,A_Ctet,Bsngl,A_OH)
angEnv2 = smirky.AngleChemicalEnvironment( A_Ctet,B_any,A_Ctet,Bsngl,A_Ctet)
angEnv3 = smirky.AngleChemicalEnvironment( A_Ctet,B_any,A_Ctet,BnoRing,A_Ctet)
print( angEnv3.asSMIRKS() )

[#6X4:1]~[#6X4:2]!@[#6X4:3]


In [8]:
# specific bonds from sublists 
bondEnv1 = smirky.BondChemicalEnvironment( A_OEth,B_any,A_Ctet)
bondEnv2 = smirky.BondChemicalEnvironment( A_Ctet,BnoRing,A_Ctet)
bondEnv3 = smirky.BondChemicalEnvironment( A_H,Bsngl,A_Ctet)
print( bondEnv3.asSMIRKS() )

[H:1]-[#6X4:2]


In [9]:
# specific vdW from sublists 
atomEnv1 = smirky.AtomChemicalEnvironment( A_OEth)
atomEnv2 = smirky.AtomChemicalEnvironment( A_Ctet)
atomEnv3 = smirky.AtomChemicalEnvironment( A_H)
atomH3 = smirky.ChemicalEnvironment( '[#1:1]-[#6](-[#7X4+1])(-[#8X2H0])-[#9]')
print( atomH3.asSMIRKS() )

[#8X2H0]-[#6](-[#1:1])(-[#7X4+1])-[#9]


In [10]:
# Generic move lists without odds (initial guess), reformat below
ChemComponents = {}
ChemComponents['AtmBase'] = ['','#1','#5','#6','#7','#8','#9','#15','#16','#17','#35','#53']
ChemComponents['AtmORdecs'] = ['','X4','X3','X2','X1']
ChemComponents['AtmAndDecs'] = ['H0','+0',]
ChemComponents['BondBase'] = ['-',':','=','#','~']
ChemComponents['BondANDDecs'] = ['@','!@','!#']
                                 
# AlkEthOH move lists (initial guess)

In [11]:
# reformat ChemComponents dict as tuples representing odds for each item
#for key in ChemComponents.keys():
#    print('ChemComponentsWithOdds[\'%s\'] = [' % key)
#    for item in ChemComponents[key]:
#        print(' (\'%s\', 1),' % item )
#    print(' ]')

### For Future:
### Decorators need weights, and they need to be weighted differently for different atomic elements. 

Guessed examples based on experience with organic chemistry:
    
--------
* C [ ('X4', 20), ('X3', 20), ('X2', 5), ('X1', 1), ]
* N [ ('X4', 10), ('X3', 10), ('X2', 10), ('X1', 1), ]
* O [ ('X4', 0), ('X3', 1), ('X2', 20), ('X1', 20), ]
* H,F,Cl,Br,I [ don't use this decorator on this atom ]
---------
* C [ ('H0', 1), ('H1', 1), ('H2', 1), ('H3', 1) ]
* N [ ('H0', 1), ('H1', 1), ('H2', 1), ('H3', 1) ]
* O [ ('H0', 20), ('H1', 20), ('H2', 1), ('H3', 0) ]
* H,F,Cl,Br,I [ don't use this decorator on this atom ]
---------
* C [ ('-1', 1), ('+0', 50), ('+1', 0) ]
* N [ ('-1', 1), ('+0', 20), ('+1', 5) ]
* O [ ('-1', 10), ('+0', 50), ('+1', 1) ]
* H,F,Cl,Br,I [ don't use this decorator on this atom ]

In [12]:
# General
atomComponentsWithOdds = {}
atomComponentsWithOdds['Basetypes'] = [
 ('#1', 10),
 ('#5', 10),
 ('#6', 10),
 ('#7', 10),
 ('#8', 10),
 ('#9', 1),
 ('#15', 2),
 ('#16', 4),
 ('#17', 1),
 ('#35', 1),
 ('#53', 1),
 ]
atomComponentsWithOdds['ORdecs'] = [ ('', 4), 
                                       ('X4', 4), ('X3', 4), ('X2', 4), ('X1', 4), 
                                       ('H3', 1), ('H2', 1), ('H1', 1), ('H0', 1)
                                      ]
# these AtmAndDecs are dummies just to get going; real chemistry puts H-count on each atom
atomComponentsWithOdds['ANDdecs'] = [ ('H0', 1), ('+0', 1), ]

bondComponentsWithOdds = {}
bondComponentsWithOdds['Basetypes'] = [ ('-', 1), (':', 1), ('=', 1), ('#', 1), ('~', 1), ]
bondComponentsWithOdds['ORdecs'] = []
bondComponentsWithOdds['ANDdecs'] = [ ('@', 1), ('!@', 1), ('!#', 1), ]

In [13]:
# for AlkEthOH
atomComponentsWithOdds = {}
atomComponentsWithOdds['Basetypes'] = [
 ('#1', 1),
 ('#5', 0),
 ('#6', 1),
 ('#7', 0),
 ('#8', 1),
 ('#9', 0),
 ('#15', 0),
 ('#16', 0),
 ('#17', 0),
 ('#35', 0),
 ('#53', 0),
 ]
atomComponentsWithOdds['ORdecs'] = [ ('', 4), 
                                       ('X4', 4), ('X3', 4), ('X2', 4), ('X1', 4), 
                                       ('H3', 1), ('H2', 1), ('H1', 1), ('H0', 1)
                                      ]
# these AtmAndDecs are dummies just to get going; real chemistry puts H-count on each atom
atomComponentsWithOdds['ANDdecs'] = [ ('H0', 1), ('+0', 1), ]

bondComponentsWithOdds = {}
bondComponentsWithOdds['Basetypes'] = [ ('-', 1), (':', 0), ('=', 0), ('#', 0), ('~', 1), ]
bondComponentsWithOdds['ORdecs'] = []
bondComponentsWithOdds['ANDdecs'] = [ ('@', 1), ('!@', 1), ('!#', 0), ]

### The following two functions are copied from iPython notebook EnvMovesTypesAndWeights:
###  movesWithWeightsFromOdds and PickMoveItemWithProb

In [14]:
# function copied from EnvMovesTypesAndWeights
def movesWithWeightsFromOdds( MovesWithOdds):
    '''Processes a dictionary of movesWithOdds (lists of string/integer tuples)
    into a dictionary of movesWithWeights usable to perform weighted
    random choices with numpy's random.choice() function.
    Argument:   a MovesWithOdds dictionary of lists of string/integer tuples
    Returns: a MovesWithWeights dictionary of pairs of a moveType-list with a 
            probabilites-list, the latter used by numpy's random.choice() function.'''
    movesWithWeights = {}
    for key in MovesWithOdds.keys():
        moves = [ item[0] for item in MovesWithOdds[key] ]
        odds =  [ item[1] for item in MovesWithOdds[key] ]
        weights = odds/np.sum(odds)
        #print( key, moves, odds, weights)
        movesWithWeights[key] = ( moves, weights)
    return movesWithWeights

In [15]:
# function copied from EnvMovesTypesAndWeights
def PickMoveItemWithProb( moveType, moveWithWeights):
    '''Picks a moveItem based on a moveType and a dictionary of moveTypes with associated probabilities
       Arguments:
         moveType: string corresponding to a key in the moveWithWeights dictionary, e.g. atomTor
         moveWithWeights: a dictionary based on moveType keys which each point to a list of probabilites
           associated with the position in the list
        Returns:
          the randomly-chosen position in the list, based on the probability, together with the probability'''
    listOfIndexes = range(0, len( moveWithWeights[moveType][1]) )
    listIndex = np.random.choice(listOfIndexes, p= moveWithWeights[moveType][1])
    return moveWithWeights[moveType][0][listIndex], moveWithWeights[moveType][1][listIndex]

In [16]:
#print( ChemComponentsWithWeights)
atomComponentsWithWeights = movesWithWeightsFromOdds(atomComponentsWithOdds)
#print( 'atomComponentsWithWeights', atomComponentsWithWeights)
bondComponentsWithWeights = movesWithWeightsFromOdds(bondComponentsWithOdds)
#print( 'bondComponentsWithWeights', bondComponentsWithWeights)
masterComponentsWithWeights = { 'atom':atomComponentsWithWeights,
                                'bond':bondComponentsWithWeights}
#print( 'masterComponentsWithWeights', masterComponentsWithWeights)

In [17]:
torsion = torsionEnv4
print( torsion.asSMIRKS() )
print( torsion.atom1.getORtypes() )
print( torsion.atom1.getANDtypes() )
atom1 = torsion.selectAtom( 4)
print('Selected atom : ',atom1.index, atom1.getORtypes(), atom1.getANDtypes() )
atmlist = torsion.getAtoms()
for atom in atmlist:
    print(atom.index, atom.getORtypes(), atom.getANDtypes() )
bond2 = torsion.getBond(torsion.atom2,torsion.atom3)
print( bond2.getORtypes() )
print( bond2.getANDtypes() )

[#8X2;H0;+0:1]~[#6X4:2]-[#8:3]~[*:4]
['#8X2']
['H0', '+0']
Selected atom :  4 [] []
3 ['#8'] []
1 ['#8X2'] ['H0', '+0']
4 [] []
2 ['#6X4'] []
['-']
[]


In [18]:
def EnvMoveIsWellFormed( moveDict, msg=''):
    '''Checks moveSeq (dict of proposed chem env micro-moves) to see if
    it is well-formed (i.e. before even looking at the chemical graph),
    returning True unless there is an obvious problem.
    Arguments: 
        moveSeq: a dict of strings constituting proposed micro-moves of a chem env move.
        msg: an informative message about the nature of the problem.
    Returns True unless it finds a problem'''
    if moveDict['action']=='joinAtom':
        if moveDict['atomOrBond']=='bond':
            msg = 'cannot join another atom to an existing bond'
            return False
        elif moveDict['ANDorOR']=='ANDtype':
            msg = 'can only join another atom as an ORtype'
            return False
    return True

In [19]:
moveDictFilename = 'moveTrees.uniq.VdW.txt'
moveDictFile = open(moveDictFilename)
moveDictdb = []
for line in moveDictFile:
    fields = line.split()
    prob = fields[0]
    moveDict = { 'action' : fields[1],
               'atomOrBond' : fields[2],
               'whichAtmBnd' : fields[3],
               'ANDorOR' : fields[4],
              }
    if EnvMoveIsWellFormed(moveDict):
        moveDictdb.append( (moveDict, prob) )
    #print( moveDict, prob)
moveDictFile.close()

In [20]:
print( len(moveDictdb))
for moveDict in moveDictdb:
    print( str(EnvMoveIsWellFormed(moveDict[0])), moveDict[0], moveDict[1])

20
True {'action': 'add', 'atomOrBond': 'atom', 'whichAtmBnd': 'atom1', 'ANDorOR': 'ANDtype'} 0.005165
True {'action': 'add', 'atomOrBond': 'atom', 'whichAtmBnd': 'atom1', 'ANDorOR': 'ORtype'} 0.015496
True {'action': 'add', 'atomOrBond': 'atom', 'whichAtmBnd': 'unIndexed', 'ANDorOR': 'ANDtype'} 0.051653
True {'action': 'add', 'atomOrBond': 'atom', 'whichAtmBnd': 'unIndexed', 'ANDorOR': 'ORtype'} 0.154959
True {'action': 'add', 'atomOrBond': 'bond', 'whichAtmBnd': 'unIndexed', 'ANDorOR': 'ANDtype'} 0.005682
True {'action': 'add', 'atomOrBond': 'bond', 'whichAtmBnd': 'unIndexed', 'ANDorOR': 'ORtype'} 0.017045
True {'action': 'delete', 'atomOrBond': 'atom', 'whichAtmBnd': 'atom1', 'ANDorOR': 'ANDtype'} 0.005165
True {'action': 'delete', 'atomOrBond': 'atom', 'whichAtmBnd': 'atom1', 'ANDorOR': 'ORtype'} 0.015496
True {'action': 'delete', 'atomOrBond': 'atom', 'whichAtmBnd': 'unIndexed', 'ANDorOR': 'ANDtype'} 0.051653
True {'action': 'delete', 'atomOrBond': 'atom', 'whichAtmBnd': 'unIndexe

In [21]:
def GetNwxfragMoveAtom( moveDict, Nwxfrag):
    atomIdxStr = moveDict['whichAtmBnd'][4]
    if atomIdxStr!='d':
        return Nwxfrag.selectAtom( int(atomIdxStr))
    indexedAtoms = Nwxfrag.getIndexedAtoms()
    unIndexedAtoms = []
    atoms = Nwxfrag.getAtoms()
    for tryatom in atoms:
        if tryatom not in indexedAtoms:
            unIndexedAtoms.append( tryatom)
    if len(unIndexedAtoms)>1:
        return np.random.choice( unIndexedAtoms)
    elif len(unIndexedAtoms)==1:
        return unIndexedAtoms[0]
    else:
        return None

In [22]:
param = atomH3
moveDict = moveDictdb[3][0]
print( param.asSMIRKS(), moveDict )
for key in moveDict.keys():
    print( key, ':', moveDict[key] )
changeAtom = GetNwxfragMoveAtom( moveDict, param)
print( changeAtom.index, changeAtom.getORtypes(), changeAtom.getANDtypes() )

[#8X2H0]-[#6](-[#1:1])(-[#7X4+1])-[#9] {'action': 'add', 'atomOrBond': 'atom', 'whichAtmBnd': 'unIndexed', 'ANDorOR': 'ORtype'}
action : add
atomOrBond : atom
whichAtmBnd : unIndexed
ANDorOR : ORtype
None ['#9'] []


In [23]:
def GenerateNwxfragORtype( ComponentsWithWeights):
    basetypeList = ComponentsWithWeights['Basetypes'][0]
    #print( 'Basetypes:', ComponentsWithWeights['Basetypes'][0])
    #print( 'Basetypes weights:', ComponentsWithWeights['Basetypes'][1])
    if len(basetypeList)<1:
        return None
    newBasetype, prob = PickMoveItemWithProb( 'Basetypes', ComponentsWithWeights)
    cumProb = prob
    
    ORtypeList = ComponentsWithWeights['ORdecs'][0]
    #print( 'ORdecs:', ORtypeList)
    #print( 'ORdecs weights:', ComponentsWithWeights['ORdecs'][1])
    if len(ORtypeList)<1:
        return newBasetype, prob
    newORdecorator, prob = PickMoveItemWithProb( 'ORdecs', ComponentsWithWeights)
    newORtype = newBasetype + newORdecorator
    cumProb *= prob
    return newORtype, prob

In [24]:
def GenerateNwxfragANDtype( ComponentsWithWeights):
    ANDtypeList = ComponentsWithWeights['ANDdecs'][0]
    #print( 'ANDdecs:', ANDtypeList)
    #print( 'ANDdecs weights:', ComponentsWithWeights['ANDdecs'][1])
    if len(ANDtypeList)<1:
        return None
    return PickMoveItemWithProb( 'ANDdecs', ComponentsWithWeights)

In [25]:
# test GenerateNwxfragORtype and GenerateNwxfragANDtype
nSamples = 3
for i in range(0,nSamples):
    print( 'atom ORtypes:', GenerateNwxfragORtype( masterComponentsWithWeights['atom']) )
for i in range(0,nSamples):
    print( 'atom ANDtypes:', GenerateNwxfragANDtype( masterComponentsWithWeights['atom']) )
for i in range(0,nSamples):
    print( 'bond ORtypes:', GenerateNwxfragORtype( masterComponentsWithWeights['bond']) )
for i in range(0,nSamples):
    print( 'bond ANDtypes:', GenerateNwxfragANDtype( masterComponentsWithWeights['bond']) )

atom ORtypes: ('#8X4', 0.16666666666666666)
atom ORtypes: ('#8X4', 0.16666666666666666)
atom ORtypes: ('#8X2', 0.16666666666666666)
atom ANDtypes: ('+0', 0.5)
atom ANDtypes: ('H0', 0.5)
atom ANDtypes: ('+0', 0.5)
bond ORtypes: ('-', 0.5)
bond ORtypes: ('-', 0.5)
bond ORtypes: ('~', 0.5)
bond ANDtypes: ('@', 0.5)
bond ANDtypes: ('@', 0.5)
bond ANDtypes: ('@', 0.5)


In [26]:
def GetNwxfragChangeling( moveDict, Nwxfrag):
    idxStr = moveDict['whichAtmBnd'][4]
    atomOrBond = moveDict['atomOrBond']
    if atomOrBond=='atom':
        if idxStr!='d':
            return Nwxfrag.selectAtom( int(idxStr))
        # begin section to randomly pick an unIndexed atom; hopefully there will be a function for this
        indexedAtoms = Nwxfrag.getIndexedAtoms()
        unIndexedAtoms = []
        atoms = Nwxfrag.getAtoms()
        for tryatom in atoms:
            if tryatom not in indexedAtoms:
                unIndexedAtoms.append( tryatom)
        if len(unIndexedAtoms)>1:
            return np.random.choice( unIndexedAtoms)
        elif len(unIndexedAtoms)==1:
            return unIndexedAtoms[0]
        # end section to randomly pick an unIndexed atom; hopefully there will be a function for this
    if atomOrBond=='bond':
        bondIdxToAtoms = { 'bond1' : (1,2), 'bond1' : (2,3), 'bond3' : (3,4),}
        if idxStr!='d':
            firstAtom = Nwxfrag.selectAtom( bondIdxToAtoms[moveDict['whichAtmBnd']][0] )
            secondAtom = Nwxfrag.selectAtom( bondIdxToAtoms[moveDict['whichAtmBnd']][1] )
            return Nwxfrag.selectBond( firstAtom, secondAtom)
        # begin section to randomly pick an unIndexed bond; hopefully there will be a function for this
        bonds = Nwxfrag.getBonds()
        unIndexedBonds = []
        for bond in bonds:
            if bond[0].index==None or bond[1].index==None:
                unIndexedBonds.append( bond )
        if len(unIndexedBonds)==1:
            return unIndexedBonds[0][2]
        elif len(unIndexedBonds)>1:
            listIdx = range( 0, len(unIndexedBonds) )
            choice = np.random.choice( listIdx)
            return unIndexedBonds[choice][2]
        # end section to randomly pick an unIndexed bond; hopefully there will be a function for this
        return None
    else:
        return None

In [27]:
# test GetNwxfragChangeling and AddToNwxfrag
startParam = atomH3
param = copy.deepcopy( startParam)
moveDict = moveDictdb[5][0]
print( param.asSMIRKS(), moveDict )
#for key in moveDict.keys():
#    print( key, ':', moveDict[key] )
changeling = GetNwxfragChangeling( moveDict, param)
#print( 'changeling properties:', changeling.index, changeling.getORtypes(), changeling.getANDtypes() )
print( 'changeling properties:', changeling.getORtypes(), changeling.getANDtypes() )

[#6](-[#1:1])(-[#7X4+1])(-[#8X2H0])-[#9] {'action': 'add', 'atomOrBond': 'bond', 'whichAtmBnd': 'unIndexed', 'ANDorOR': 'ORtype'}
changeling properties: ['-'] []


In [28]:
def ProposeNewType( moveDict, ComponentsWithWeights):
    #print( 'ProposeNewType moveDict:', moveDict )
    atomOrBond = moveDict['atomOrBond']
    if moveDict['ANDorOR']=='ORtype':
        newPair = GenerateNwxfragORtype( ComponentsWithWeights[atomOrBond])
    elif moveDict['ANDorOR']=='ANDtype':
        newPair = GenerateNwxfragANDtype( ComponentsWithWeights[atomOrBond])
    return newPair

In [29]:
# test ProposeNewType
for moveDict in moveDictdb:
    print( moveDict[0]['atomOrBond'], moveDict[0]['ANDorOR'], ':',
         ProposeNewType( moveDict[0], masterComponentsWithWeights))

atom ANDtype : ('+0', 0.5)
atom ORtype : ('#6X4', 0.16666666666666666)
atom ANDtype : ('+0', 0.5)
atom ORtype : ('#1', 0.16666666666666666)
bond ANDtype : ('!@', 0.5)
bond ORtype : ('-', 0.5)
atom ANDtype : ('+0', 0.5)
atom ORtype : ('#1X2', 0.16666666666666666)
atom ANDtype : ('+0', 0.5)
atom ORtype : ('#1X1', 0.16666666666666666)
bond ANDtype : ('!@', 0.5)
bond ORtype : ('~', 0.5)
atom ORtype : ('#8X1', 0.16666666666666666)
atom ORtype : ('#8', 0.16666666666666666)
atom ANDtype : ('H0', 0.5)
atom ORtype : ('#1X4', 0.16666666666666666)
atom ANDtype : ('+0', 0.5)
atom ORtype : ('#6H2', 0.041666666666666664)
bond ANDtype : ('@', 0.5)
bond ORtype : ('-', 0.5)


In [30]:
def IsActionViable( moveDict, existing, proposal, msg):
    action = moveDict['action']
    #print('IsActionViable:  action:',action,'; existing types:',existing,'; proposed newType:',proposal)
    if action=='add':
        if proposal=='None':
            print('proposed newType is null; will not add')
            return False
        if proposal in existing:
            print('proposed newType is in existing list; will not add')
            return False
    elif action=='delete':
        # ToDo: check to make sure not completely deleting whole atom which connects at least two others
        if len(existing)<1:
            #msg = 'existing list is already empty; will not delete'
            #print('IsActionViable: msg is', msg)
            print('existing list is already empty; will not delete')
            return False
        if len(existing)==1 and moveDict['whichAtmBnd'][4]!='d':
            print('will not delete indexed atom or bond with only one type')
            return False
        if len(existing)==1 and moveDict['atomOrBond']=='bond':
            print('will not delete last bond joining two atoms')
            return False
    elif action=='swap':
        if len(existing)<1 or proposal=='None':
            print('missing one of existing type or newType; will not swap')
            return False
        if len(existing)==1 and existing[0]==proposal:
            print('Swapping identical types is pointless; will not swap')
            return False
    elif action=='joinAtom':
        # ToDo: insert valence check (don't join to atom with full valence)
        # ToDo: insert subst position check (don't join to beta substituent)
        if len(existing)<1 or proposal=='None':
            print('missing one of existing atom or new atom; will not join atom')
            return False
    return True

In [31]:
def SetNwxfragNewList( moveDict, changeling, newList):
    #print( 'SetNwxfragNewList newList:', newList )
    if moveDict['ANDorOR']=='ORtype':
        #print( 'SetNwxfragNewList old type:', changeling.getORtypes() )
        changeling.setORtypes( newList)
        #print( 'SetNwxfragNewList new type:', changeling.getORtypes() )
        return True
    elif moveDict['ANDorOR']=='ANDtype':
        #print( 'SetNwxfragNewList old type:', changeling.getANDtypes() )
        changeling.setANDtypes( newList)
        #print( 'SetNwxfragNewList new type:', changeling.getANDtypes() )
        return True
    else:
        return False

In [32]:
def EffectMoveOnNwxfragList( moveDict, frag, changeling, existing, proposal):
    action = moveDict['action']
    #print('EffectMoveOnNwxfragList:  action:',action,'; existing types:',existing,'; proposed newType:',proposal)
    newList = existing
    #
    # begin section for action delete and swap (swap is delete followed by add)
    # begin with special case for actually removing an atom completely, ie delete the last atom ORtype
    if action=='delete' and len(newList)==1 and moveDict['atomOrBond']=='atom'and moveDict['ANDorOR']=='ORtype':
        # for time being just remove the whole thing; worry about detailed balance later
        onlyEmpty = False
        frag.removeAtom( changeling, onlyEmpty)
        return True
    # now the more general case for removing the last type from a list: make an empty list
    if action=='delete' or action=='swap':
        if len(newList)==1:
            newList = []
    # now to remove a random item from a list of more than one
        else:
            listOfIndexes = range(0, len( newList) )
            idxToDelete = np.random.choice(listOfIndexes)
            #print('EffectMoveOnNwxfragList delete index %d from list', newList)
            del newList[idxToDelete]
            #print('EffectMoveOnNwxfragList list after delete is', newList)
    #
    # begin section for action add and swap (swap is delete followed by add)
    if action=='add' or action=='swap':
        newList.append( proposal)
        #print('EffectMoveOnNwxfragList about to set newList:', newList)
        if not SetNwxfragNewList( moveDict, changeling, newList):
            print('EffectMoveOnNwxfragList newList failed:', newList)
            return False
    #
    return True

In [33]:
def EffectJoinNewNwxfragAtom( moveDict, frag, changeling, existing, proposal):
    print('EffectJoinNewNwxfragAtom: attaching [', proposal,'] to existing atom', existing)
    frag.addAtom( changeling, ['~'], None, [proposal], None, None)
    return True

In [34]:
def MoveNwxfrag( frag, moveDict, weightdParts, msg=''):
    '''Modifies an OpenForcefield Networkx molecular fragment by applying a move described by
    a moveDict (dict of proposed chem env micro-moves), based on weighted random choices of
    chemical components in weightdParts. If the proposed move is not viable, it returns False.
    Arguments:
        frag: an OpenForcefield Networkx molecular fragment (representing a force field parameter)
        moveDict: a list of strings constituting a sequence of proposed chem env micro-moves
        weightdParts: dictionary of chemical components with probabilities of being chosen
        msg: an informative message about the nature of the problem
    Returns True unless it finds a problem'''
    #print( frag.asSMIRKS(), moveDict )
    
    # stage 1: get the atom or bond we are going to work on
    atomOrBond = moveDict['atomOrBond']
    changeling = GetNwxfragChangeling( moveDict, param)
    if changeling==None:
        msg = 'MoveNwxfrag: changeling is None, returning false'
        return False
    #print( 'MoveNwxfrag changeling:', changeling.getORtypes(), changeling.getANDtypes() )
    
    # stage 2: get the current list we need to modify
    ANDorOR = moveDict['ANDorOR']
    if ANDorOR=='ORtype':
        currlist = changeling.getORtypes()
        #print('MoveNwxfrag got ORtypes:', currlist)
    elif ANDorOR=='ANDtype':
        currlist = changeling.getANDtypes()
        #print('MoveNwxfrag got ANDtypes:', currlist)
    else:
        msg = 'MoveNwxfrag: could not retrieve requested ORtypes or ANDtypes'
        return False
    
    # stage 3: get the component list and associated weights
    proposed = ProposeNewType( moveDict, masterComponentsWithWeights)
    if proposed==None:
        msg = 'MoveNwxfrag: proposed (newType, probability) is None, returning false'
        return False
    #print('MoveNwxfrag proposed (newType probability):', proposed)
    
    #stage 4: Test viability of desired action
    mesg = ''
    IsViable = IsActionViable( moveDict, currlist, proposed[0], mesg)
    if not IsViable:
        msg = 'MoveNwxfrag: proposal is not viable, returning false. Details:\n'+mesg
        #print('MoveNwxfrag returning msg:', msg)
        return False
    #print('MoveNwxfrag proposed action %s is viable ' % moveDict['action'])
    
    #stage 5: perform action
    if moveDict['action']!='joinAtom':
        EffectMoveOnNwxfragList( moveDict, frag, changeling, currlist, proposed[0])
    else:
        EffectJoinNewNwxfragAtom( moveDict, frag, changeling, currlist, proposed[0])
    
    return True

In [35]:
# test MoveNwxfrag
startParam = atomH3
param = copy.deepcopy( startParam)
moveDict = moveDictdb[13][0]
#print( param.asSMIRKS(), moveDict )
msg = ''
if not MoveNwxfrag( param, moveDict, masterComponentsWithWeights, msg):
    print('move failed, message is:', msg)
else:
    print('move worked, new SMIRKS is:', param.asSMIRKS())

EffectJoinNewNwxfragAtom: attaching [ #6X1 ] to existing atom ['#7X4+1']
move worked, new SMIRKS is: [#6](-[#1:1])(-[#7X4+1]~[#6X1])(-[#8X2H0])-[#9]


In [36]:
# test MoveNwxfrag
startParam = atomH3
param = copy.deepcopy( startParam)
print( 'Starting with', param.asSMIRKS())
#print( len(moveDictdb) )
for i, moveDict in enumerate( moveDictdb):
    param = copy.deepcopy( startParam)
    moves =  moveDict[0]
    print( '\nworking on %2d %8s %4s %10s %7s :' % (i, moves['action'], moves['atomOrBond'], moves['whichAtmBnd'], moves['ANDorOR']))
    if not MoveNwxfrag( param, moves, masterComponentsWithWeights, msg):
        print('  Failed')
    else:
        print('  Succeeded : %s' % ( param.asSMIRKS()))

Starting with [#8X2H0]-[#6](-[#1:1])(-[#7X4+1])-[#9]

working on  0      add atom      atom1 ANDtype :
  Succeeded : [#8X2H0]-[#6](-[#1;H0:1])(-[#7X4+1])-[#9]

working on  1      add atom      atom1  ORtype :
  Succeeded : [#7X4+1]-[#6](-[#1,#8:1])(-[#8X2H0])-[#9]

working on  2      add atom  unIndexed ANDtype :
  Succeeded : [#7X4+1]-[#6](-[#1:1])(-[#8X2H0])-[#9;H0]

working on  3      add atom  unIndexed  ORtype :
  Succeeded : [#6,#6H2](-[#1:1])(-[#7X4+1])(-[#8X2H0])-[#9]

working on  4      add bond  unIndexed ANDtype :
  Succeeded : [#1:1]-[#6](-;@[#7X4+1])(-[#8X2H0])-[#9]

working on  5      add bond  unIndexed  ORtype :
  Succeeded : [#8X2H0]-[#6](-,~[#1:1])(-[#7X4+1])-[#9]

working on  6   delete atom      atom1 ANDtype :
existing list is already empty; will not delete
  Failed

working on  7   delete atom      atom1  ORtype :
will not delete indexed atom or bond with only one type
  Failed

working on  8   delete atom  unIndexed ANDtype :
existing list is already empty; will 